In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [2]:
def get_page(link = '/soccer'):
    page_link = 'http://www.betexplorer.com'+link    
    try:
        # fetch the content from url
        page_response = requests.get(page_link, timeout=5)
        # parse html
        page_content = BeautifulSoup(page_response.content, "html.parser")
    except:
        #caso a página não responda, espera por 3min e tenta novamente
        time.sleep(180)
        page_response = requests.get(page_link, timeout=5)
        # parse html
        page_content = BeautifulSoup(page_response.content, "html.parser")
    return page_content


def scrapGroupPhase(page, page_link,country, tournament, season):
    #df_goals = pd.DataFrame(columns=['Match_Id','Team_Id', 'Time','Player_Name']) #para coleta d
    df = pd.DataFrame(columns=['Country','Tournament', 'Season','Round','Home', 'Away','Score','Bet_Home','Bet_Drawn','Bet_Away','Game_Date','Game_Details','Fase'])
    #div = page_content.find_all('ul', attrs = {'class' : 'list-tabs list-tabs--secondary'})
    th_position = 0
    td_position = 0
    data = []
    for content in page:
        for li in content.find_all('li'):            
            fase = li.text
            rodada =''
            try: 
                link = li.select('a')[0].attrs['href']
                page_content = get_page(page_link+link)
            except:
                page_content = get_page(page_link)
            
            try:
                table = page_content.find_all('table')[0]
                for row in table.find_all(['td', 'th']):
                    if row.name == 'th' and th_position == 0:
                        rodada = row.text
                        data = [country, tournament, season, rodada]
                        th_position = 1
                    elif row.name == 'td':
                        if th_position == 0:
                            if rodada == '':
                                rodada = fase
                            data = [country, tournament, season, rodada]
                            th_position = 1
                        if td_position == 0:
                            data.append(row.text.split(' - ')[0])
                            data.append(row.text.split(' - ')[1])
                            for a in row.find_all('a', href=True):
                                link = a['href']
                                #df_goals = scrapGoals(len(df),[row.text.split(' - ')[0],row.text.split(' - ')[1]],link, df_goals)
                        elif 2 <= td_position <= 4:
                            try:
                                data.append(row.select('span')[2].attrs['data-odd'])
                            except:
                                try: 
                                    data.append(row.attrs['data-odd'])
                                except:
                                    data.append(row.text)                    
                        else:
                            data.append(row.text)
                        td_position+=1       
                        if td_position == 6:
                            data.append(link)
                            data.append(fase)
                            df.loc[len(df)] = data
                            data = [country, tournament, season, rodada]
                            th_position = 0
                            td_position = 0
            except:
                continue
    #return df, df_goals
    return df

In [3]:
def scrapChampionship(page_content, country, tournament, season):
    #df_goals = pd.DataFrame(columns=['Match_Id','Team_Id', 'Time','Player_Name'])
    df = pd.DataFrame(columns=['Country','Tournament', 'Season','Round','Home', 'Away','Score','Bet_Home','Bet_Drawn','Bet_Away','Game_Date','Game_Details','Fase'])
    th_position = 0
    td_position = 0
    rodada = ''
    data = [country,tournament, season, rodada]
    table = page_content.find_all('table')[0]
    
    for row in table.find_all(['td', 'th']):
        if row.name == 'th' and th_position == 0:
            rodada = row.text
            data = [country, tournament, season, rodada]
            th_position = 1
        elif row.name == 'td':
            if td_position == 0:
                data.append(row.text.split(' - ')[0])
                data.append(row.text.split(' - ')[1])
                for a in row.find_all('a', href=True):
                    link = a['href']
                    #df_goals = scrapGoals(len(df),[row.text.split(' - ')[0],row.text.split(' - ')[1]],link, df_goals)
            elif 2 <= td_position <= 4:
                try:
                    data.append(row.select('span')[2].attrs['data-odd'])
                except:
                    try: 
                        data.append(row.attrs['data-odd'])
                    except:
                        data.append(row.text)                    
            else:
                data.append(row.text)
            td_position+=1       
            if td_position == 6:
                data.append(link)
                data.append('')
                df.loc[len(df)] = data
                data = [country, tournament, season, rodada]
                th_position = 0
                td_position = 0
    #return df, df_goals
    return df

def scrapGoals(match_id, team_id, page_link, df_goals):
    page_content = get_page(page_link)
    for ul in page_content.find_all('ul', attrs = {'class' : 'list-details list-details--shooters'}):
        i = 0
        for li in ul.findAll('li'):
            if len(li) > 1:
                for tr in li.findAll('tr'):
                    #home team
                    if i == 0:
                        df_goals.loc[len(df_goals)] = [match_id, team_id[0], tr.text.split('.')[0],tr.text.split('.')[1]]
                    #away team
                    else:
                        df_goals.loc[len(df_goals)] = [match_id, team_id[1], tr.text.split('.')[0],tr.text.split('.')[1]]
            i = 1

    return df_goals

In [4]:
df_tournaments_links = pd.read_csv('Tournament Links.csv')
log_columns = list(df_tournaments_links.columns)
log_columns.append('Function')
log_columns.append('Error')
log = pd.DataFrame(columns= log_columns)
df_tournaments_links.Link = df_tournaments_links.Link+'results/'

In [6]:
df_tournaments_links=df_tournaments_links[(df_tournaments_links.Link == '/soccer/indonesia/liga-2/results/')]
df_tournaments_links.Link = '/soccer/indonesia/liga-2-2017/'

In [21]:
current_country = df_tournaments_links.loc[df_tournaments_links.index.min()].Country
df_country_competitions = pd.DataFrame(columns=['Country','Tournament', 'Season','Round','Home', 'Away','Score','Bet_Home','Bet_Drawn','Bet_Away','Game_Date','Game_Details','Fase'])
for row in df_tournaments_links.values:
    row = list(row)
    page_link = row[3]
    try:
        page_content = get_page(page_link)
    except:
        row.append('get_page')
        row.append('Timeout')
        log.loc[len(log)] = row
        continue
        
    div = page_content.find_all('ul', attrs = {'class' : 'list-tabs list-tabs--secondary'})
    try:
        if len(div) > 0:
            row.append('scrapGroupPhase')
            df = scrapGroupPhase(div, page_link,row[0], row[1], row[2])
            #df, df_goals = scrapGroupPhase(div, page_link)
        else:
            row.append('scrapChampionship')
            #df, df_goals = scrapChampionship(page_content)
            df = scrapChampionship(page_content,row[0], row[1], row[2])
        #if len(df_goals) > 0:
            #df_goals.to_excel('data/'+row[1]+' - '+row[2].split('/')[0]+' - Goals.xlsx', index=False)
        df_country_competitions = df_country_competitions.append(df)
        #if row[0] == current_country:
        #    df_country_competitions = df_country_competitions.append(df)
        #else:
        #    df_country_competitions.drop_duplicates(['Country','Tournament','Season','Home','Away','Score','Bet_Home',
        #                                             'Bet_Drawn','Bet_Away','Game_Date','Game_Details'],inplace=True)
        #    df_country_competitions.to_excel('data/a'+current_country+'.xlsx', index=False)
        #    #df_country_competitions.to_excel('data/teste.xlsx', index=False)
        #    df_country_competitions = df
        #    current_country = row[0]
    except Exception as e:        
        row.append(str(e))
        log.loc[len(log)] = row
        print(row)
df_country_competitions.drop_duplicates(['Country','Tournament','Season','Home','Away','Score','Bet_Home',
                                         'Bet_Drawn','Bet_Away','Game_Date','Game_Details'],inplace=True)
df_country_competitions.reset_index(inplace=True,drop=True)
df_country_competitions.to_csv('Games_Results2.csv',encoding='utf-8',index_label='GameID')
log.to_csv('alog2.csv', encoding='utf-8', index=False)

In [22]:
df_country_competitions

,Country,Tournament,Season,Round,Home,Away,Score,Bet_Home,Bet_Drawn,Bet_Away,Game_Date,Game_Details,Fase
0,Indonesia,Liga 2,2017,Final,PSMS Medan,Persebaya,2:3 ET,2.47,3.18,2.48,28.11.2017,/soccer/indonesia/liga-2-2017/psms-medan-perse...,Next matches
1,Indonesia,Liga 2,2017,3rd place,PSIS Semarang,Martapura,6:4 ET,1.97,3.51,3.11,28.11.2017,/soccer/indonesia/liga-2-2017/psis-semarang-ma...,Next matches
2,Indonesia,Liga 2,2017,Semi-finals,Persebaya,Martapura,3:1,1.82,3.46,3.65,25.11.2017,/soccer/indonesia/liga-2-2017/persebaya-martap...,Next matches
3,Indonesia,Liga 2,2017,Semi-finals,PSMS Medan,PSIS Semarang,2:0 ET,2.47,3.17,2.66,25.11.2017,/soccer/indonesia/liga-2-2017/psms-medan-psis-...,Next matches
4,Indonesia,Liga 2,2017,Group 5,Martapura,Madiun Putra,4:0,1.30,5.09,6.64,14.09.2017,/soccer/indonesia/liga-2-2017/martapura-madiun...,First round
5,Indonesia,Liga 2,2017,Group 5,Persebaya,Pesinga Ngawi,4:0,,,,14.09.2017,/soccer/indonesia/liga-2-2017/persebaya-pesing...,First round
6,Indonesia,Liga 2,2017,Group 5,Persepam Madura,PSIM Yogyakarta,1:1,1.51,3.96,4.95,14.09.2017,/soccer/indonesia/liga-2-2017/persepam-madura-...,First round
7,Indonesia,Liga 2,2017,Group 5,PSBI Blitar,Persatu Tuban,1:0,,,,14.09.2017,/soccer/indonesia/liga-2-2017/psbi-blitar-pers...,First round
8,Indonesia,Liga 2,2017,Group 3,Persibas Banyumas,Persip P.,3:1,1.12,6.75,12.33,13.09.2017,/soccer/indonesia/liga-2-2017/persibas-banyuma...,First round
9,Indonesia,Liga 2,2017,Group 3,PSCS Cilacap,PSS Sleman,2:0,2.02,4.01,2.73,13.09.2017,/soccer/indonesia/liga-2-2017/pscs-cilacap-pss...,First round
